In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
import pandas as pd
data=pd.read_csv('/kaggle/input/iris/Iris.csv')
print(data.head())
print(data.dtypes)
data.drop('Id',axis=1,inplace=True)
print(data.head())
print(data['Species'].unique())
print(data.isnull().sum())

In [ ]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
data['Spesies']=le.fit_transform(data['Species'])
target=data['Spesies']
datax=data.drop(['Species','Spesies'],axis=1)
print(data)
print(datax)

In [ ]:
from sklearn.cluster import KMeans
km=KMeans(n_clusters=5)
km.fit(datax)
datax['cluster']=km.predict(datax)
print(datax)

In [ ]:
clustered=pd.concat([datax,pd.get_dummies(datax['cluster'])],axis=1)
print(clustered)
clustered.drop('cluster',axis=1,inplace=True)
print(clustered)

In [ ]:
for i in [0,1,2,3,4]:
    print('Untuk cluster:',i)
    ax=clustered[['SepalLengthCm','SepalWidthCm','PetalLengthCm','PetalWidthCm']][clustered[i]==1]
    print(ax.describe())

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
ss=StandardScaler()
rand=RandomForestClassifier()

Xclust=ss.fit_transform(clustered)
X=ss.fit_transform(datax)
xtrainss,xtestss,ytrainss,ytestss=train_test_split(X,target,test_size=0.3,random_state=3)
xctrainss,xctestss,yctrainss,yctestss=train_test_split(Xclust,target,test_size=0.3,random_state=3)
for k in [5,6,7,8,9,10]:
    kn=KNeighborsClassifier(n_neighbors=k)
    print("k=",k)
    print("Normal")
    kn.fit(xtrainss,ytrainss)
    print(kn.score(xtestss,ytestss))
    print("Clustered")
    kn.fit(xctrainss,yctrainss)
    print(kn.score(xctestss,yctestss))

    

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import plot_confusion_matrix
kn=KNeighborsClassifier(n_neighbors=8)

kn.fit(xtrainss,ytrainss)
plot_confusion_matrix(kn,xtestss,ytestss)
plt.title("KNN Normal")
kn.fit(xctrainss,yctrainss)
plot_confusion_matrix(kn,xctestss,yctestss)
plt.title("KNN Clustered")

rand.fit(xtrainss,ytrainss)
plot_confusion_matrix(rand,xtestss,ytestss)
plt.title("Randomforest Normal")
rand.fit(xctrainss,yctrainss)
plot_confusion_matrix(rand,xctestss,yctestss)
plt.title("Randomforest Clustered")

